[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/function-calling/nvidia/nvidia-query-agent-tool.ipynb)

## Weaviate Query Agent with NVIDIA NIM

This notebook will show you how to define the Weaviate Query Agent as a tool through NVIDIA NIM.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
1. NVIDIA NIM API key. Grab one [here](https://build.nvidia.com/models).
1. Install the Weaviate Agents package with `pip install weaviate-agents`
1. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [ ]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import json

from openai import OpenAI

In [6]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["NVIDIA_API_KEY"] = ""

### Define Query Agent function

In [ ]:
def send_query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.

    Args:
        query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

    Returns:
        str: The response from the database containing relevant information.
    """

    # connect to your Weaviate Cloud instance
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"), 
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        headers={ # add the API key to the model provider from your Weaviate collection, for example `headers={"X-Nvidia-Api-Key": os.getenv("NVIDIA_API_KEY")}` 
        }
    )

    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["Blogs"] 
    )
    return query_agent.run(query).final_answer

### Define the tool JSON

In [ ]:
query_agent_tool = {
    "type": "function",
    "function": {
        "name": "send_query_agent_request",
        "description": "Send a query to the database and get the response.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Any question or query to search for in the database"
                }
            },
            "required": ["query"]
        }
    }
}

### Connect to NVIDIA client

In [ ]:
client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = os.getenv("NVIDIA_API_KEY"),
)

MODEL_NAME = "meta/llama-3.1-70b-instruct"

### Define function calling loop

In [22]:
def function_calling_loop(query, tool_definition, query_function, model_name="meta/llama-3.1-70b-instruct"):
    """
    Execute a function calling loop with a specified tool and query function.
    
    Args:
        query (str): The user query to process
        tool_definition (dict): The JSON definition of the tool to be used for function calling
        query_function (callable): The function to execute when the tool is called
        model_name (str): The LLM model to use for function calling
        
    Returns:
        str: The final response from the model
    """
    # Initialize messages with user query
    messages = [
        {"role": "user", "content": query}
    ]
    
    # First call - model decides whether to use the tool
    chat_response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=[tool_definition],
        tool_choice="auto",
        stream=False
    )
    
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    
    if assistant_message.tool_calls and len(assistant_message.tool_calls) > 0:
        tool_call = assistant_message.tool_calls[0]
        tool_name = tool_call.function.name
        
        args = json.loads(tool_call.function.arguments)
        
        print(f"Tool called: {tool_name}")
        print(f"Arguments: {args}")
        
        # Execute the query function with the provided arguments
        if tool_name == tool_definition["function"]["name"]:
            # For a QueryAgent tool, we expect a 'query' parameter
            if 'query' in args:
                result = query_function(args['query'])
            else:
                # Handle other possible argument structures
                result = query_function(**args)
            
            # Ensure the result is a string
            if not isinstance(result, str):
                if isinstance(result, dict) and 'final_answer' in result:
                    result_str = result['final_answer']
                else:
                    result_str = json.dumps(result)
            else:
                result_str = result
            
            # Add tool response to messages
            messages.append({
                "role": "tool", 
                "content": result_str, 
                "tool_call_id": tool_call.id, 
                "name": tool_name
            })
            
            # Second call - model generates final response based on tool output
            chat_response = client.chat.completions.create(
                model=model_name,
                messages=messages,
                tools=[tool_definition],
                tool_choice="auto",
                stream=False
            )
            
            assistant_message = chat_response.choices[0].message
    
    # Print and return the final response
    if assistant_message.content:
        print(f"\nFinal response: {assistant_message.content}")
        return assistant_message.content
    else:
        print("\nNo text response from assistant.")
        return None

### Query time

In [23]:
# Example usage in a Jupyter notebook cell
response = function_calling_loop(
    query="What is Docker?", 
    tool_definition=query_agent_tool,
    query_function=send_query_agent_request
)

Tool called: send_query_agent_request
Arguments: {'query': 'What is Docker?'}

Final response: Docker is a set of platform-as-a-service products that use operating-system-level virtualization to deliver software in packages called containers. Containers are a method of virtualization of the operating system to isolate applications into separate environments.
